# Advent of Code 2017

In [123]:
import csv
import numpy as np
from itertools import product
import math
import re
import pandas as pd

## Day 1

In [2]:
def d1p1(inp):
    seq1 = [int(inp[i:(i+1)]) for i in range(len(inp))] + [int(inp[-1])]
    return sum([seq1[i] for i in range(len(seq1)-1) if (seq1[i] - seq1[i+1]) == 0])

In [3]:
def d1p2(inp):
    seq2 = [int(inp[i:(i+1)]) for i in range(len(inp))] * 2
    step = int(len(inp) / 2)
    return sum([seq2[i] for i in range(len(inp)) if (seq2[i] - seq2[i+step]) == 0])

### Result

In [4]:
with open('input_day1.txt', 'r') as f:
    inp = f.read()
print(f"==> Part1: {d1p1(inp)}")
print(f"==> Part2: {d1p2(inp)}")

==> Part1: 1216
==> Part2: 1072


## Day 2

In [5]:
def d2p1(inp):
    return sum(inp.max(1) - inp.min(1))

In [6]:
def d2p2(inp):
    def evenly_div(arr):
        pairs = np.asarray(list(product(arr, repeat=2)))
        div = (pairs[:, 0] / pairs[:, 1]).astype(int)
        return div[(pairs[:, 0] % pairs[:, 1] == 0) & (pairs[:, 0] != pairs[:, 1])][0]
    return sum(np.apply_along_axis(evenly_div, 1, inp))

### Result

In [7]:
with open('input_day2.txt', 'r') as f:
    inp = [l for l in csv.reader(f)]
inp = np.asarray(inp).astype(int)
print(f"==> Part1: {d2p1(inp)}")
print(f"==> Part2: {d2p2(inp)}")

==> Part1: 34925
==> Part2: 221


## Day 3

In [8]:
def d3_prereq(inp):
    upper_sqrt = math.ceil(math.sqrt(inp))
    edge_len = upper_sqrt + int(upper_sqrt % 2 == 0)
    return upper_sqrt, edge_len

In [9]:
def d3p1(inp):
    upper_sqrt, edge_len = d3_prereq(inp)
    print(f'square root \t= {math.sqrt(inp)} (=> {upper_sqrt} => {edge_len})')

    ring_dist = int(edge_len / 2)
    print(f'ring to center \t= {ring_dist}')

    last_num = edge_len**2
    print(f'max of ring \t= {last_num}')

    edge_center = np.array([(last_num - int(edge_len / 2)) - (edge_len - 1) * i for i in range(4)])
    edge_dist = min(abs(edge_center - inp))
    print(f'to edge center = {edge_dist}')

    return ring_dist + edge_dist

In [10]:
def d3p2(inp):
    _, edge_len = d3_prereq(inp)
    grid_size = edge_len + 2
    grid = np.zeros([grid_size, grid_size])
    center = int(grid_size / 2)
    x, y = center, center
    val = 1
    grid[y, x] = val

    while val < inp:
        if grid[y-1, x] == 0 and grid[y, x-1] == 0 and grid[y+1, x] == 0 and grid[y, x+1] == 0:
            x += 1
        # UP if: u == 0 and l != 0
        elif grid[y-1, x] == 0 and grid[y, x-1] != 0:
            y -= 1
        # LEFT if: l == 0 and d != 0
        elif grid[y, x-1] == 0 and grid[y+1, x] != 0:
            x -= 1
        # DOWN if: d == 0 and r != 0
        elif grid[y+1, x] == 0 and grid[y, x+1] != 0:
            y += 1
        # RIGHT if: r == 0 and u != 0
        elif grid[y, x+1] == 0 and grid[y-1, x] != 0:
            x += 1
        val = sum([
            grid[y+1, x],
            grid[y-1, x],
            grid[y, x+1],
            grid[y, x-1],
            grid[y+1, x+1],
            grid[y+1, x-1],
            grid[y-1, x+1],
            grid[y-1, x-1]        
        ])
        grid[y, x] = val
    return int(val)

### Result

In [11]:
inp = 361527
print(f"==> Part1: {d3p1(inp)}")
print(f"==> Part2: {d3p2(inp)}")

square root 	= 601.2711534740379 (=> 602 => 603)
ring to center 	= 301
max of ring 	= 363609
to edge center = 25
==> Part1: 326
==> Part2: 363010


## Day 4

In [12]:
def d4p1(inp):
    return sum([len(p) == len(set(p)) for p in inp])

In [13]:
def d4p2(inp):
    ana = [[''.join(sorted(set(w))) for w in p] for p in inp]
    return sum([len(p) == len(set(p)) for p in ana])

### Result

In [14]:
with open('input_day4.txt', 'r') as f:
    inp = f.readlines()
inp = [phrase.rstrip('\n').split(' ') for phrase in inp]

print(f"==> Part1: {d4p1(inp)}")
print(f"==> Part2: {d4p2(inp)}")

==> Part1: 466
==> Part2: 251


## Day 5

In [24]:
def d5(inp, p1=True):
    pos, n_steps = 0, 0
    while 0 <= pos < len(inp):
        offset = inp[pos]
        inp[pos] += 1 if (offset < 3 or p1) else -1
        pos = pos+offset
        n_steps += 1
    return n_steps, inp

### Result

In [29]:
inp = []
with open('input_day5.txt', 'r') as f:
    line = f.readline()
    while len(line) > 0:
        inp.append(int(line.rstrip('\n')))
        line = f.readline()
        
p1_result, p1_state = d5(inp, p1=True)
p2_result, _ = d5(inp, p1=False)
print(f"==> Part1: {p1_result}")
print(f"==> Part2: {p2_result}")

==> Part1: 325922
==> Part2: 35


## Day 6

In [17]:
def d6p1(state):
    seen_states = []
    n_cycles = 0
    while str(state) not in seen_states:
        ix = np.argmax(state)
        n_blocks = state[ix]
        seen_states.append(str(state))
        state[ix] = 0
        while n_blocks > 0:
            ix += 1
            ix *= int(ix < len(state))
            n_blocks -= 1
            state[ix] += 1
        n_cycles += 1
    return n_cycles, state

In [18]:
def d6p2(state):
    n_cycles = 0
    loop_state = str(state)
    while n_cycles == 0 or str(state) != loop_state:
        ix = np.argmax(state)
        n_blocks = state[ix]
        state[ix] = 0
        while n_blocks > 0:
            ix += 1
            ix *= int(ix < len(state))
            n_blocks -= 1
            state[ix] += 1
        n_cycles += 1
    return n_cycles

### Result

In [30]:
with open('input_day6.txt', 'r') as f:
    inp = f.read().rstrip('\n').split('\t')
inp = np.asarray(inp).astype(int)

p1_result, p1_state = d6p1(inp)
p2_result = d6p2(p1_state)
print(f"==> Part1: {p1_result}")
print(f"==> Part2: {p2_result}")

==> Part1: 7864
==> Part2: 1695


## Day 7

In [66]:
with open('input_day7.txt', 'r') as f:
    inp = f.readlines()
inp = np.asarray([re.findall('^(\w+) \((\d+)\)(?: -> )?(.*)$',
                             i.rstrip('\n'))[0] for i in inp])
print(inp[:4])

[['rmhcw' '26' '']
 ['dmmriu' '61' '']
 ['zjyaaoc' '89' 'khekb, lywwhf']
 ['kjgaib' '385' '']]


For every node find the parent node

In [215]:
parents = dict()
for ele in inp:
    if len(ele[2]) > 0:
        children = ele[2].split(', ')
        for child in children:
            parents[child] = ele[0]

The only node without a parent is the __LUCA__ _(last universal common ancestor)_

In [216]:
luca = (set(inp[:, 0]) - set(parents.keys())).pop()
luca

'ykpsek'

### 7.2

In [276]:
class DF_Tree:
    def __init__(self, data, parents):
        df = pd.DataFrame(data, columns=['id', 'weight', 'children'])
        df.weight = df.weight.astype(int)
        df.set_index('id', inplace=True)
        df['sub_weights'] = 0
        self.df = df.join(pd.Series(parents, name='parent'))
        
    def walk(self, node_id):
        node = self.df.loc[node_id]
        if len(node.children) == 0:
            return int(node.weight)
        sub_w = sum([self.walk(child_id) for child_id in node.children.split(', ')])
        self.df.loc[node_id, 'sub_weights'] = sub_w
        return self.df.loc[node_id, 'sub_weights'] + int(node.weight)

tree = DF_Tree(inp, parents)
tree.walk(luca)

471343

In [277]:
print(tree.df.shape)
tree.df.head()

(1416, 4)


,weight,children,sub_weights,parent
id,,,,
rmhcw,26,,0,bajivws
dmmriu,61,,0,wbyisd
zjyaaoc,89,"khekb, lywwhf",156,zacca
kjgaib,385,,0,nhfqb
nvvdii,28,,0,atyrf


In [282]:
tree.df = tree.df.assign(total_weight = tree.df.sub_weights + tree.df.weight)
tree.df.head()

,weight,children,sub_weights,parent,total_weight
id,,,,,
rmhcw,26,,0,bajivws,26
dmmriu,61,,0,wbyisd,61
zjyaaoc,89,"khekb, lywwhf",156,zacca,245
kjgaib,385,,0,nhfqb,385
nvvdii,28,,0,atyrf,28


In [294]:
child_weight_std = tree.df.groupby('parent')['total_weight'].agg(np.std)
unbalanced = child_weight_std.loc[child_weight_std > 0].index.tolist()

In [301]:
tree.df.loc[tree.df.parent.isin(unbalanced)].sort_values('parent')

,weight,children,sub_weights,parent,total_weight
id,,,,,
irqwcab,8461,"hxsausc, qtmdrsq, xfqfzsh",825,rsalq,9286
uduyfo,1710,"amhaz, cumah, zqfvypo, dribos",7585,rsalq,9295
sdxbol,2587,"qwsjrs, dnqva, ddssmn",6699,rsalq,9286
jiaiwto,936,"ezftzv, uwcumu, dylatux, fhezp, lcbmlo",8350,rsalq,9286
viydtj,10,"kstjj, qybpmcs, qpiwrz, koordg",9276,rsalq,9286
cumah,1069,"pqmyjtl, wczux, rfjilj",834,uduyfo,1903
zqfvypo,1810,"juxyuv, ednozov",84,uduyfo,1894
amhaz,39,"pvnsq, vlxrpve, fchnaw, toejq, dnxvlw, iqjhw, ...",1855,uduyfo,1894
dribos,1213,"cjdmv, lpozpnx, jknws",681,uduyfo,1894


In [304]:
1069 - (1903-1894)

1060